In [1]:
# from gensim.corpora import Dictionary
from glob import glob
import nltk 
from nltk import tokenize
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import re
from time import time
from unicodedata import normalize
#import database as db

In [2]:
#ruta
#filelist = glob('/home/jhon/proyectos/natural_language_processing/entes_de_control/resolucion_entes/entes_de_control_test/part1/*.txt')
filelist = glob('/home/jhonex/Documentos/proyectos/natural_language_processing/entes_de_control/resolucion_entes/*.txt')
#lista 
letters = []
for file in filelist:
    #
    with open(file, 'r') as txtfile: 
        letters.append((tokenize.word_tokenize(txtfile.read()), re.match('.*/(.*)-.*', file).group(1)))
letters = pd.DataFrame(letters, columns=['Text', 'Name'])
letters = pd.DataFrame([(np.concatenate(letters[letters.Name == name].Text.values), name) 
                        for name in letters.Name.unique()], columns=['Text', 'Name'])
letters

,Text,Name
0,"[¡, ., DNP…“, :, W, TODOS, POR, UN, ., =, —, '...",RESOLUCIрN APELACIрN


In [14]:
def cleanDocs(letters):
    start = time()
    clean_letters = []
    for _, letter in letters.iterrows():
        words = [normalize('NFKD', word.lower()).encode('ascii', 'ignore').decode('utf-8') for word in letter['Text']]
        # is alphabetic character, not is a stopword, not is a custom word, has at least three letters, 
        # has at least one vowel, has at least one consonant
        words = [word for word in words if word not in stopwords.words('spanish')]
        clean_letters.append(' '.join(words))

    print('Cleaning took %.2f seconds' % (time() - start))
    return pd.DataFrame(clean_letters)

dup = letters.insert(1, 'CleanText', cleanDocs(letters))


Cleaning took 1.26 seconds


ValueError: cannot insert CleanText, already exists

In [ ]:
#stopwords.words('spanish')

In [15]:
letters.CleanText[letters.Name == 'RESOLUCIрN APELACIрN'].values[0]

" . dnp... : w . =  ''  ''  '' nuevo pais 0 . ' paz equidad educacion gb superservrcros - - * supenntendencia servicios  publicos domiciliarios ' |i|illlllllillllllllill|lllll|lllillillllllllllllllllllll illllllllllll ' pagina 1de4 resolucionno . sspd - 201,68200320445 09/12/2016 expediente . 2016820390116861 decide recurso apelacion director territorial norte superintendencia servicios publicos domiciliarios superintendencia servicios publicos ejercicio facultades , especial confiere articulo 79 numeral 29 , articulos 154 159 ley 142 1994 , modificada ley 689 2001 ; articulo 20 numeral 1 2 decreto 990 2002 ; ley 1437 2011 ; fundamento siguientes antecedentes mediante escrito radicado . re9510201614209 05 julio 2016 , ( ) senor ( ) alfredo brochado ortega , identificado ( ) codigo suscriptor . 2099214 , presento recurso reposicion subsidio apelacion decision . 3995142 15 ' unio 2016 , proferida empresa electrificadora caribe s. a. e. s. p.- electricaribe s. a. e. s. p.- jose garcia san

In [16]:
def context(target_word, tar_passage, left_margin = 10, right_margin = 10):
    tokens = tokenize.word_tokenize(tar_passage)
    text = nltk.Text(tokens)
    c = nltk.ConcordanceIndex(text.tokens, key = lambda s: s.lower())
    concordance_txt = [text.tokens[offset - left_margin : offset + right_margin]
                       for offset in c.offsets(target_word)]
    return [''.join([x + ' ' for x in con_sub]) for con_sub in concordance_txt]

In [17]:
cleanText = re.sub('[^\w\s]', '', letters[letters.Name == 'RESOLUCIрN APELACIрN'].CleanText.values[0])
val1=re.findall('(\d{16})', cleanText)
val2=re.findall('sspd\s*(\d{14})', cleanText)

numero=set(re.findall('\s(\d{14})\s', cleanText))
val3=set(numero)-set(val2)

#print(set(re.findall('re\s*(\d{13})', cleanText)))
print(val1)
print(val2)
print(val3)
resoluciones = ['requerir', 'apertura de investigacion', 'sancion', 'archivar', 'confirmar', 'revocar', 
                'rechazar', 'tramitar', 'improcedente', 'rechazo', 'procedente', 'modificar', 'inhibirse', 
                'no acceder', 'corregir', 'aclarar']
for resolucion in resoluciones:
    coincidence = re.findall(resolucion + ' ', cleanText)
    if coincidence:
        print(coincidence)
        
results = context('modificar', cleanText, 0, 10)

for result in results:
    print(result)
#print(re.fidall('sancion', cleanText))
if results == [] :
            print('\n:(\n"no se se detecto contexto" ') 

['2016820390116861']
['20168200320445']
{'20168201060262'}
['modificar ']
modificar decision administrativa 3995142 15 junio 2016 proferida electrificadora caribe 


In [18]:
letters

,Text,CleanText,Name
0,"[¡, ., DNP…“, :, W, TODOS, POR, UN, ., =, —, '...",. dnp... : w . = '' '' '' nuevo pais 0 . '...,RESOLUCIрN APELACIрN


In [10]:
print(cleanText)

electriqaribe  ntewenidq superservctos barranquilla  20 junio 2017 senores  superintendencia servicios publicos domiciliarios attn   dra  jenny elizabeth lindo directora territorial norte carrera 59  75  134 ciudad asunto  remision expediente tramite recurso apelacion contrato no7577626  radicado   re1110201703179   folio  s  32  mediante presente cumplimiento articulo 159 ley 142 1994 damos traslado expediente contentivo decision empresarial contrato abajo descritos surta recurso apelacion despacho  atentamente  xt sidney gomez guerrero central escritos hyepezcod1738798 aenor  aenor  h   roma    nu  n am  mu   electriqaribe  ntewenidq superservctos barranquilla  20 junio 2017 senores  superintendencia servicios publicos domiciliarios attn   dra  jenny elizabeth lindo directora territorial norte carrera 59  75  134 ciudad asunto  remision expediente tramite recurso apelacion contrato no7577626  radicado   re1110201703179   folio  s  32  mediante presente cumplimiento articulo 159 ley 1

In [15]:
set([7,8,9,0,6,7,8,5])-set([5,3,7,9,2,3,4,2,3])
le

{0, 6, 8}

### RNN model

In [9]:
#dic = Dictionary(letters.CleanText.apply(lambda text: tokenize.word_tokenize(text)).values)
#dic.token2id

In [11]:
#test = letters[letters.index == 8].CleanText.apply(lambda text: tokenize.word_tokenize(text)).values[0]
#dic.doc2bow(['servicios', 'nuevo', 'pais', 'publicos', 'domiciliarios'])
#test

In [32]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

tokenizer = Tokenizer(nb_words=1000)
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print(one_hot_results[0])

Using TensorFlow backend.


Found 9 unique tokens.
[ 0.  1.  0.  0.  1.  1.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.